In [3]:
# 自动计算cell的计算时间
%load_ext autotime

%matplotlib inline
%config InlineBackend.figure_format='svg' #矢量图设置，让绘图更清晰

time: 402 ms (started: 2021-08-26 00:09:32 +08:00)


In [ ]:
%%bash

# 增加更新
git add *.ipynb *.md

git remote -v

git commit -m '更新 #2  Aug 26, 2021'

#git push origin master
git push

In [5]:
#设置使用的gpu
import tensorflow as tf
from tensorflow import keras

gpus = tf.config.list_physical_devices("GPU")

if gpus:
   
    gpu0 = gpus[1] #如果有多个GPU，仅使用第0个GPU
    tf.config.experimental.set_memory_growth(gpu0, True) #设置GPU显存用量按需使用
    # 或者也可以设置GPU显存为固定使用量(例如：4G)
    #tf.config.experimental.set_virtual_device_configuration(gpu0,
    #    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]) 
    tf.config.set_visible_devices([gpu0],"GPU")

time: 154 ms (started: 2021-08-26 00:10:15 +08:00)


# 介绍

Keras 模型由多个组件组成：
* 架构或配置，指定模型包含哪些层以及它们如何连接。
* 一组权重值（“模型状态”）。
* 优化器（通过编译模型定义）。
* 一组损失和指标（通过编译模型或调用 add_loss() 或 add_metric() 定义）。

Keras API 可以一次将所有这些部分保存到磁盘，或者仅选择性地保存其中的一些：
* 将所有内容以 TensorFlow SavedModel 格式（或较旧的 Keras H5 格式）保存到单个存档中。 这是标准做法。
* 仅保存架构/配置，通常为 JSON 文件。
* 仅保存权重值。 这通常在训练模型时使用。

让我们来看看这些选项中的每一个。 你什么时候会使用其中一种，它们是如何工作的？

# 如何保存和加载模型

如果您只有 10 秒钟的时间阅读本指南，那么您需要了解以下内容。

**保存一个keras模型：**

In [ ]:
model = ...  # Get model (Sequential, Functional Model, or Model subclass)
model.save('path/to/location')

**重新加载模型：**

In [ ]:
from tensorflow import keras
model = keras.models.load_model('path/to/location')

现在，让我们来看看细节。

# 设置

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

# 全模型保存和加载

您可以将整个模型保存到单个工件中。 它将包括：
* 模型的架构/配置
* 模型的权重值（在训练期间学习）
* 模型的编译信息（如果 compile() 被调用）
* 优化器及其状态（如果有）（这使您可以从离开的地方重新开始训练）

## APIs

* `model.save()` 或 `tf.keras.models.save_model()`
* `tf.keras.models.load_model()`

您可以使用两种格式将整个模型保存到磁盘：TensorFlow SavedModel 格式和较旧的 Keras H5 格式。 推荐的格式是 SavedModel。 它是您使用 `model.save()` 时的默认设置。

您可以通过以下方式切换到 H5 格式：
* 将 save_format='h5' 传递给 save()。
* 将以 .h5 或 .keras 结尾的文件名传递给 save()。

## 保存模型格式

SavedModel 是更全面的保存格式，用于保存模型架构、权重和调用函数的跟踪 Tensorflow 子图。 这使 Keras 能够恢复内置层和自定义对象。

**例子：**

In [6]:
def get_model():
    # Create a simple model.
    inputs = keras.Input(shape=(32,))
    outputs = keras.layers.Dense(1)(inputs)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer="adam", loss="mean_squared_error")
    return model


model = get_model()

# Train the model.
test_input = np.random.random((128, 32))
test_target = np.random.random((128, 1))
model.fit(test_input, test_target)

# Calling `save('my_model')` creates a SavedModel folder `my_model`.
model.save("my_model")

# It can be used to reconstruct the model identically.
reconstructed_model = keras.models.load_model("my_model")

# Let's check:
np.testing.assert_allclose(
    model.predict(test_input), reconstructed_model.predict(test_input)
)

# The reconstructed model is already compiled and has retained the optimizer
# state, so training can resume:
reconstructed_model.fit(test_input, test_target)

4/4 [==============================] - 2s 5ms/step - loss: 0.7806
INFO:tensorflow:Assets written to: my_model/assets
4/4 [==============================] - 0s 5ms/step - loss: 0.6838


time: 4.68 s (started: 2021-08-26 00:18:17 +08:00)


## SavedModel 包含的内容

调用 model.save('my_model') 创建一个名为 my_model 的文件夹，其中包含以下内容：

In [7]:
!ls my_model

assets	keras_metadata.pb  saved_model.pb  variables
time: 205 ms (started: 2021-08-26 00:19:18 +08:00)


模型架构和训练配置（包括优化器、损失和指标）存储在 saved_model.pb 中。 权重保存在 variables/ 目录中。

有关 SavedModel 格式的详细信息，请参阅 SavedModel 指南（磁盘上的 SavedModel 格式）。

## SavedModel 如何处理自定义对象

保存模型及其层时，SavedModel 格式存储类名、调用函数、损失和权重（以及配置，如果实现）。 call 函数定义了模型/层的计算图。

在没有模型/层配置的情况下，调用函数用于创建一个与原始模型一样存在的模型，该模型可以被训练、评估和用于推理。

尽管如此，在编写自定义模型或层类时定义 get_config 和 from_config 方法始终是一个好习惯。 这允许您稍后在需要时轻松更新计算。 有关更多信息，请参阅有关自定义对象的部分。

例子：

In [8]:
class CustomModel(keras.Model):
    def __init__(self, hidden_units):
        super(CustomModel, self).__init__()
        self.hidden_units = hidden_units
        self.dense_layers = [keras.layers.Dense(u) for u in hidden_units]

    def call(self, inputs):
        x = inputs
        for layer in self.dense_layers:
            x = layer(x)
        return x

    def get_config(self):
        return {"hidden_units": self.hidden_units}

    @classmethod
    def from_config(cls, config):
        return cls(**config)


model = CustomModel([16, 16, 10])
# 通过调用来构建模型
input_arr = tf.random.uniform((1, 5))
outputs = model(input_arr)
model.save("my_model")

# 选项 1：使用 custom_object 参数加载。
loaded_1 = keras.models.load_model(
    "my_model", custom_objects={"CustomModel": CustomModel}
)


# 选项 2：在没有 CustomModel 类的情况下加载。

# 删除自定义模型类，确保加载器无权访问。
del CustomModel

loaded_2 = keras.models.load_model("my_model")
np.testing.assert_allclose(loaded_1(input_arr), outputs)
np.testing.assert_allclose(loaded_2(input_arr), outputs)

print("Original model:", model)
print("Model Loaded with custom objects:", loaded_1)
print("Model loaded without the custom object class:", loaded_2)

INFO:tensorflow:Assets written to: my_model/assets
Original model: <__main__.CustomModel object at 0x7f5e9c042ac0>
Model Loaded with custom objects: <__main__.CustomModel object at 0x7f5e5c18d1f0>
Model loaded without the custom object class: <tensorflow.python.keras.saving.saved_model.load.CustomModel object at 0x7f5e5c18d580>
time: 598 ms (started: 2021-08-26 00:26:59 +08:00)


第一个加载的模型是使用 config 和 CustomModel 类加载的。 第二个模型是通过动态创建与原始模型类似的模型类来加载的。

## 配置 SavedModel

TensoFlow 2.4 中的新功能 参数 `save_traces` 已添加到 `model.save`，它允许您切换 SavedModel 函数跟踪。 保存函数是为了允许 Keras 在没有原始类定义的情况下重新加载自定义对象，因此当 `save_traces=False` 时，所有自定义对象都必须定义 `get_config/from_config` 方法。 加载时，自定义对象必须传递给 `custom_objects` 参数。 `save_traces=False` 减少 SavedModel 使用的磁盘空间并节省时间。

## keras H5格式

Keras 还支持保存单个 HDF5 文件，其中包含模型的架构、权重值和 compile() 信息。 它是 SavedModel 的轻量级替代品。

**例子：**

In [9]:
model = get_model()

# Train the model.
test_input = np.random.random((128, 32))
test_target = np.random.random((128, 1))
model.fit(test_input, test_target)

# Calling `save('my_model.h5')` creates a h5 file `my_model.h5`.
model.save("my_h5_model.h5")

# It can be used to reconstruct the model identically.
reconstructed_model = keras.models.load_model("my_h5_model.h5")

# Let's check:
np.testing.assert_allclose(
    model.predict(test_input), reconstructed_model.predict(test_input)
)

# The reconstructed model is already compiled and has retained the optimizer
# state, so training can resume:
reconstructed_model.fit(test_input, test_target)

4/4 [==============================] - 0s 5ms/step - loss: 0.2847


time: 622 ms (started: 2021-08-26 00:33:37 +08:00)


## 限制

与 SavedModel 格式相比，H5 文件中没有包含两件事：
* 通过 model.add_loss() 和 model.add_metric() 添加的外部损失和指标不会被保存（与 SavedModel 不同）。 如果您的模型有这样的损失和指标，并且您想恢复训练，则需要在加载模型后自行添加这些损失。 请注意，这不适用于通过 self.add_loss() 和 self.add_metric() 在层内创建的损失/指标。 只要层被加载，这些损失和指标就会被保留，因为它们是层调用方法的一部分。
* 自定义图层等自定义对象的计算图不包含在保存的文件中。 在加载时，Keras 将需要访问这些对象的 Python 类/函数以重建模型。 请参阅自定义对象。

# 保存架构

模型的配置（或架构）指定模型包含哪些层，以及这些层如何连接*。 如果您有模型的配置，那么可以使用权重的新初始化状态创建模型，并且没有编译信息。

> 请注意，这仅适用于使用功能或顺序 API 而非子类模型定义的模型。

## 顺序模型或功能 API 模型的配置
这些类型的模型是层的显式图：它们的配置始终以结构化形式提供。

## APIs
* get_config() 和 from_config()
* tf.keras.models.model_to_json() 和 tf.keras.models.model_from_json()

### get_config() 和 from_config()
调用 config = model.get_config() 将返回一个包含模型配置的 Python 字典。 然后可以通过 Sequential.from_config(config)（对于 Sequential 模型）或 Model.from_config(config)（对于 Functional API 模型）重建相同的模型。

相同的工作流程也适用于任何可序列化层。

图层示例：

In [15]:
layer = keras.layers.Dense(3, activation="relu")
layer_config = layer.get_config()
new_layer = keras.layers.Dense.from_config(layer_config)

time: 4.23 ms (started: 2021-08-26 00:39:46 +08:00)


In [16]:
layer_config

{'name': 'dense_11',
 'trainable': True,
 'dtype': 'float32',
 'units': 3,
 'activation': 'relu',
 'use_bias': True,
 'kernel_initializer': {'class_name': 'GlorotUniform',
  'config': {'seed': None}},
 'bias_initializer': {'class_name': 'Zeros', 'config': {}},
 'kernel_regularizer': None,
 'bias_regularizer': None,
 'activity_regularizer': None,
 'kernel_constraint': None,
 'bias_constraint': None}

time: 3.8 ms (started: 2021-08-26 00:39:50 +08:00)


顺序模型示例：

In [13]:
model = keras.Sequential([keras.layers.InputLayer((32,)), keras.layers.Dense(1)])
config = model.get_config()
new_model = keras.Sequential.from_config(config)

time: 46.1 ms (started: 2021-08-26 00:39:26 +08:00)


In [14]:
config

{'name': 'sequential_1',
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 32),
    'dtype': 'float32',
    'sparse': False,
    'ragged': False,
    'name': 'input_4'}},
  {'class_name': 'Dense',
   'config': {'name': 'dense_10',
    'trainable': True,
    'dtype': 'float32',
    'units': 1,
    'activation': 'linear',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'GlorotUniform',
     'config': {'seed': None}},
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'kernel_regularizer': None,
    'bias_regularizer': None,
    'activity_regularizer': None,
    'kernel_constraint': None,
    'bias_constraint': None}}]}

time: 4.66 ms (started: 2021-08-26 00:39:31 +08:00)


Functional模型例子：

In [17]:
inputs = keras.Input((32,))
outputs = keras.layers.Dense(1)(inputs)
model = keras.Model(inputs, outputs)
config = model.get_config()
new_model = keras.Model.from_config(config)

time: 38.4 ms (started: 2021-08-26 00:41:17 +08:00)


In [18]:
config

{'name': 'model_2',
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 32),
    'dtype': 'float32',
    'sparse': False,
    'ragged': False,
    'name': 'input_5'},
   'name': 'input_5',
   'inbound_nodes': []},
  {'class_name': 'Dense',
   'config': {'name': 'dense_12',
    'trainable': True,
    'dtype': 'float32',
    'units': 1,
    'activation': 'linear',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'GlorotUniform',
     'config': {'seed': None}},
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'kernel_regularizer': None,
    'bias_regularizer': None,
    'activity_regularizer': None,
    'kernel_constraint': None,
    'bias_constraint': None},
   'name': 'dense_12',
   'inbound_nodes': [[['input_5', 0, 0, {}]]]}],
 'input_layers': [['input_5', 0, 0]],
 'output_layers': [['dense_12', 0, 0]]}

time: 5.33 ms (started: 2021-08-26 00:41:22 +08:00)


### to_json() 和 tf.keras.models.model_from_json()

这与 get_config / from_config 类似，不同之处在于它将模型转换为 JSON 字符串，然后可以在没有原始模型类的情况下加载该字符串。 它也特定于模型，不适用于图层。

例子：

In [19]:
model = keras.Sequential([keras.Input((32,)), keras.layers.Dense(1)])
json_config = model.to_json()
new_model = keras.models.model_from_json(json_config)

time: 47.9 ms (started: 2021-08-26 00:42:42 +08:00)


In [20]:
json_config

'{"class_name": "Sequential", "config": {"name": "sequential_2", "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [null, 32], "dtype": "float32", "sparse": false, "ragged": false, "name": "input_6"}}, {"class_name": "Dense", "config": {"name": "dense_13", "trainable": true, "dtype": "float32", "units": 1, "activation": "linear", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}]}, "keras_version": "2.5.0", "backend": "tensorflow"}'

time: 3.06 ms (started: 2021-08-26 00:42:48 +08:00)


## 自定义对象

## 模型和层

子类模型和层的架构在方法 `__init__` 和 `call` 中定义。 它们被视为 Python 字节码，无法序列化为与 JSON 兼容的配置——您可以尝试序列化字节码（例如通过 pickle），但它完全不安全，意味着您的模型无法加载到不同的系统上。

为了使用自定义层或子类模型保存/加载模型，您应该覆盖 `get_config` 和可选的 `from_config` 方法。 此外，您应该使用 register 自定义对象，以便 Keras 知道它。

## 自定义函数

自定义函数（例如激活丢失或初始化）不需要 get_config 方法。 只要将其注册为自定义对象，函数名称就足以加载。

## 仅加载 TensorFlow 图

可以加载由 Keras 生成的 TensorFlow 图。 如果这样做，则不需要提供任何 custom_objects。 你可以这样做：

In [21]:
model.save("my_model")
tensorflow_graph = tf.saved_model.load("my_model")
x = np.random.uniform(size=(4, 32)).astype(np.float32)
predicted = tensorflow_graph(x).numpy()

INFO:tensorflow:Assets written to: my_model/assets
time: 315 ms (started: 2021-08-26 00:46:02 +08:00)


请注意，此方法有几个缺点： 
* 出于可追溯性原因，您应该始终可以访问所使用的自定义对象。 您不希望将无法重新创建的模型投入生产。 
* `tf.saved_model.load` 返回的对象不是 Keras 模型。 所以它不是那么容易使用。 例如，您将无权访问 `.predict()` 或 `.fit()`

即使不鼓励使用它，它也可以在您遇到困难时为您提供帮助，例如，如果您丢失了自定义对象的代码或在使用 `tf.keras.models.load_model()` 加载模型时遇到问题。

您可以在有关 [tf.saved_model.load](https://www.tensorflow.org/api_docs/python/tf/saved_model/load) 的页面中找到更多信息

## 定义配置方法
规格：
* `get_config` 应该返回一个 JSON 可序列化的字典，以便与 Keras 架构和模型保存 API 兼容。
* `from_config(config)` (classmethod) 应该返回一个从配置创建的新层或模型对象。 默认实现返回 `cls(**config)`。

**例子：**

In [22]:
class CustomLayer(keras.layers.Layer):
    def __init__(self, a):
        self.var = tf.Variable(a, name="var_a")

    def call(self, inputs, training=False):
        if training:
            return inputs * self.var
        else:
            return inputs

    def get_config(self):
        return {"a": self.var.numpy()}

    # 实际上没有必要在这里定义 `from_config`，因为返回 `cls(**config)` 是默认行为。
    @classmethod
    def from_config(cls, config):
        return cls(**config)


layer = CustomLayer(5)
layer.var.assign(2)

serialized_layer = keras.layers.serialize(layer)
new_layer = keras.layers.deserialize(
    serialized_layer, custom_objects={"CustomLayer": CustomLayer}
)

time: 5.93 ms (started: 2021-08-26 01:10:35 +08:00)


In [23]:
serialized_layer

{'class_name': 'CustomLayer', 'config': {'a': 2}}

time: 3.12 ms (started: 2021-08-26 01:11:29 +08:00)


## 注册自定义对象
Keras 会记录哪个类生成了配置。 从上面的例子中，tf.keras.layers.serialize 生成自定义层的序列化形式：

In [ ]:
{'class_name': 'CustomLayer', 'config': {'a': 2}}

Keras 保留了所有内置层、模型、优化器和度量类的主列表，用于找到正确的类来调用 from_config。 如果找不到该类，则会引发错误（值错误：未知层）。 有几种方法可以将自定义类注册到此列表：
* 在加载函数中设置 custom_objects 参数。 （请参阅上面“定义配置方法”部分中的示例）
* tf.keras.utils.custom_object_scope 或 tf.keras.utils.CustomObjectScope
* tf.keras.utils.register_keras_serializable

## 自定义层和功能示例

In [25]:
class CustomLayer(keras.layers.Layer):
    def __init__(self, units=32, **kwargs):
        super(CustomLayer, self).__init__(**kwargs)
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

    def get_config(self):
        config = super(CustomLayer, self).get_config()
        config.update({"units": self.units})
        return config


def custom_activation(x):
    return tf.nn.tanh(x) ** 2


# Make a model with the CustomLayer and custom_activation
inputs = keras.Input((32,))
x = CustomLayer(32)(inputs)
outputs = keras.layers.Activation(custom_activation)(x)
model = keras.Model(inputs, outputs)


# Retrieve the config
config = model.get_config()

# At loading time, register the custom objects with a `custom_object_scope`:
custom_objects = {"CustomLayer": CustomLayer, "custom_activation": custom_activation}
with keras.utils.custom_object_scope(custom_objects):
    new_model = keras.Model.from_config(config)

time: 87.5 ms (started: 2021-08-26 01:24:35 +08:00)


## 内存模型克隆

您还可以通过 tf.keras.models.clone_model() 在内存中克隆模型。 这相当于获取配置然后从其配置重新创建模型（因此它不保留编译信息或层权重值）。

例子：

In [26]:
with keras.utils.custom_object_scope(custom_objects):
    new_model = keras.models.clone_model(model)

time: 33.6 ms (started: 2021-08-26 01:25:30 +08:00)


# 仅保存和加载模型的权重值

您可以选择仅保存和加载模型的权重。 这在以下情况下很有用：
* 您只需要用于推理的模型：在这种情况下，您不需要重新开始训练，因此您不需要编译信息或优化器状态。
* 您正在进行迁移学习：在这种情况下，您将重用先前模型的状态来训练新模型，因此您不需要先前模型的编译信息。

## 用于内存中权重传输的 API
可以使用 get_weights 和 set_weights 在不同对象之间复制权重：
* tf.keras.layers.Layer.get_weights()：返回一个 numpy 数组列表。
* tf.keras.layers.Layer.set_weights()：将模型权重设置为 weights 参数中的值。

下面的例子。

**在内存中将权重从一层转移到另一层**

In [27]:
def create_layer():
    layer = keras.layers.Dense(64, activation="relu", name="dense_2")
    layer.build((None, 784))
    return layer


layer_1 = create_layer()
layer_2 = create_layer()

# Copy weights from layer 1 to layer 2
layer_2.set_weights(layer_1.get_weights())

time: 21.3 ms (started: 2021-08-26 01:29:55 +08:00)


In [28]:
layer_1.get_weights()

[array([[ 0.07517382, -0.02564053, -0.01778287, ...,  0.04400925,
         -0.04631099, -0.01575509],
        [-0.05088177,  0.02289279, -0.06196739, ..., -0.02231796,
         -0.05191361, -0.06438279],
        [ 0.06155025,  0.03366031,  0.01924893, ...,  0.01024388,
          0.07083455, -0.08136846],
        ...,
        [-0.01094113,  0.00202128,  0.06957934, ...,  0.07453486,
          0.07569278, -0.03115816],
        [-0.08033627, -0.00596955, -0.06056705, ...,  0.03357223,
          0.02330612, -0.03764711],
        [ 0.02882098, -0.06347737, -0.07837206, ...,  0.01619513,
          0.07841682, -0.00738808]], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)]

time: 11.4 ms (started: 2021-08-26 01:30:11 +08:00)


## 在内存中将权重从一个模型转移到另一个具有兼容架构的模型

In [29]:
# 创建一个简单的函数模型
inputs = keras.Input(shape=(784,), name="digits")
x = keras.layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = keras.layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = keras.layers.Dense(10, name="predictions")(x)
functional_model = keras.Model(inputs=inputs, outputs=outputs, name="3_layer_mlp")

# 定义一个具有相同架构的子类模型
class SubclassedModel(keras.Model):
    def __init__(self, output_dim, name=None):
        super(SubclassedModel, self).__init__(name=name)
        self.output_dim = output_dim
        self.dense_1 = keras.layers.Dense(64, activation="relu", name="dense_1")
        self.dense_2 = keras.layers.Dense(64, activation="relu", name="dense_2")
        self.dense_3 = keras.layers.Dense(output_dim, name="predictions")

    def call(self, inputs):
        x = self.dense_1(inputs)
        x = self.dense_2(x)
        x = self.dense_3(x)
        return x

    def get_config(self):
        return {"output_dim": self.output_dim, "name": self.name}


subclassed_model = SubclassedModel(10)
# 调用子类模型一次以创建权重。
subclassed_model(tf.ones((1, 784)))

# 将权重从功能模型复制到子类模型。
subclassed_model.set_weights(functional_model.get_weights())

assert len(functional_model.weights) == len(subclassed_model.weights)
for a, b in zip(functional_model.weights, subclassed_model.weights):
    np.testing.assert_allclose(a.numpy(), b.numpy())

time: 89 ms (started: 2021-08-26 01:32:07 +08:00)


## 无状态层的情况

由于无状态层不会改变权重的顺序或数量，因此即使存在额外/缺失的无状态层，模型也可以具有兼容的架构。

In [30]:
inputs = keras.Input(shape=(784,), name="digits")
x = keras.layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = keras.layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = keras.layers.Dense(10, name="predictions")(x)
functional_model = keras.Model(inputs=inputs, outputs=outputs, name="3_layer_mlp")

inputs = keras.Input(shape=(784,), name="digits")
x = keras.layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = keras.layers.Dense(64, activation="relu", name="dense_2")(x)

# 添加一个不包含任何权重的 dropout 层。
x = keras.layers.Dropout(0.5)(x)
outputs = keras.layers.Dense(10, name="predictions")(x)
functional_model_with_dropout = keras.Model(
    inputs=inputs, outputs=outputs, name="3_layer_mlp"
)

functional_model_with_dropout.set_weights(functional_model.get_weights())

time: 258 ms (started: 2021-08-26 01:33:44 +08:00)


## 用于将权重保存到磁盘并重新加载它们的 API
权重可以通过以下格式调用 model.save_weights 保存到磁盘：
* TensorFlow 检查点
* HDF5

model.save_weights 的默认格式是 TensorFlow 检查点。 有两种方法可以指定保存格式：
* save_format 参数：将值设置为 save_format="tf" 或 save_format="h5"。
* 路径参数：如果路径以 .h5 或 .hdf5 结尾，则使用 HDF5 格式。 除非设置了 save_format，否则其他后缀将导致 TensorFlow 检查点。

还可以选择将权重作为内存中的 numpy 数组进行检索。 每个 API 都有其优缺点，详情如下。

## TF 检查点格式
例子：

In [31]:
# Runnable example
sequential_model = keras.Sequential(
    [
        keras.Input(shape=(784,), name="digits"),
        keras.layers.Dense(64, activation="relu", name="dense_1"),
        keras.layers.Dense(64, activation="relu", name="dense_2"),
        keras.layers.Dense(10, name="predictions"),
    ]
)
sequential_model.save_weights("ckpt")
load_status = sequential_model.load_weights("ckpt")

# `assert_consumed` 可用于验证所有变量值都已从检查点恢复。 
# 有关 Status 对象中的其他方法，请参阅 [`tf.train.Checkpoint.restore`]
# (https://www.tensorflow.org/api_docs/python/tf/train/Checkpoint/restore)。
load_status.assert_consumed()

time: 1.84 s (started: 2021-08-26 01:36:02 +08:00)


## 格式详情
TensorFlow Checkpoint 格式使用对象属性名称保存和恢复权重。 例如，考虑 `tf.keras.layers.Dense` 层。 该层包含两个权重：`dense.kernel` 和`dense.bias`。 当图层保存为 tf 格式时，生成的检查点包含键“kernel”和“bias”及其对应的权重值。 有关更多信息，请参阅 TF 检查点指南中的“加载机制”。

请注意，属性/图形边是以父对象中使用的名称命名的，而不是变量的名称。 考虑下面示例中的 CustomLayer。 变量 CustomLayer.var 与“var”一起保存为键的一部分，而不是“var_a”。

In [32]:
class CustomLayer(keras.layers.Layer):
    def __init__(self, a):
        self.var = tf.Variable(a, name="var_a")


layer = CustomLayer(5)
layer_ckpt = tf.train.Checkpoint(layer=layer).save("custom_layer")

ckpt_reader = tf.train.load_checkpoint(layer_ckpt)

ckpt_reader.get_variable_to_dtype_map()

{'save_counter/.ATTRIBUTES/VARIABLE_VALUE': tf.int64,
 '_CHECKPOINTABLE_OBJECT_GRAPH': tf.string,
 'layer/var/.ATTRIBUTES/VARIABLE_VALUE': tf.int32}

time: 29.6 ms (started: 2021-08-26 01:38:15 +08:00)


## 迁移学习示例
本质上，只要两个模型具有相同的架构，它们就能够共享相同的检查点。

例子：

In [33]:
inputs = keras.Input(shape=(784,), name="digits")
x = keras.layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = keras.layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = keras.layers.Dense(10, name="predictions")(x)
functional_model = keras.Model(inputs=inputs, outputs=outputs, name="3_layer_mlp")

# 提取设置部分中定义的功能模型的一部分。
# 以下几行生成一个新模型，该模型不包括功能模型的最终输出层。
pretrained = keras.Model(
    functional_model.inputs, functional_model.layers[-1].input, name="pretrained_model"
)
# 随机分配“训练过的”权重。
for w in pretrained.weights:
    w.assign(tf.random.normal(w.shape))
pretrained.save_weights("pretrained_ckpt")
pretrained.summary()

# 假设这是一个单独的程序，其中只有 'pretrained_ckpt' 存在。
# 创建一个具有不同输出维度的新功能模型。
inputs = keras.Input(shape=(784,), name="digits")
x = keras.layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = keras.layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = keras.layers.Dense(5, name="predictions")(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="new_model")

# 将 pretrained_ckpt 中的权重加载到模型中。
model.load_weights("pretrained_ckpt")

# Check that all of the pretrained weights have been loaded.
for a, b in zip(pretrained.weights, model.weights):
    np.testing.assert_allclose(a.numpy(), b.numpy())

print("\n", "-" * 50)
model.summary()

# 示例 2：顺序模型
# 重新创建预训练模型，并加载保存的权重。
inputs = keras.Input(shape=(784,), name="digits")
x = keras.layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = keras.layers.Dense(64, activation="relu", name="dense_2")(x)
pretrained_model = keras.Model(inputs=inputs, outputs=x, name="pretrained")

# 顺序示例：
model = keras.Sequential([pretrained_model, keras.layers.Dense(5, name="predictions")])
model.summary()

pretrained_model.load_weights("pretrained_ckpt")

# ＃ 警告！ 调用 `model.load_weights('pretrained_ckpt')` 不会抛出错误，但不会*按预期工作。 
# 如果您检查砝码，您会发现没有任何砝码已加载。 `pretrained_model.load_weights()` 是正确的调用方法。

Model: "pretrained_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
digits (InputLayer)          [(None, 784)]             0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                50240     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
Total params: 54,400
Trainable params: 54,400
Non-trainable params: 0
_________________________________________________________________

 --------------------------------------------------
Model: "new_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
digits (InputLayer)          [(None, 784)]             0         
_________________________________________________________________
dense_1 (Dense)              (None, 64) 

time: 179 ms (started: 2021-08-26 01:41:27 +08:00)


通常建议坚持使用相同的 API 来构建模型。 如果您在 Sequential 和 Functional，或 Functional 和 subclassed 等之间切换，则始终重建预训练模型并将预训练权重加载到该模型。

下一个问题是，如果模型架构完全不同，如何将权重保存并加载到不同的模型中？ 解决方案是使用 tf.train.Checkpoint 来保存和恢复确切的层/变量。

例子：

In [35]:
# 创建一个基本上使用functional_model的第一层和最后一层的子类模型。
# 首先，保存functional_model的第一个和最后一个密集层的权重。
first_dense = functional_model.layers[1]
last_dense = functional_model.layers[-1]
ckpt_path = tf.train.Checkpoint(
    dense=first_dense, kernel=last_dense.kernel, bias=last_dense.bias
).save("ckpt")

# 定义子类模型。
class ContrivedModel(keras.Model):
    def __init__(self):
        super(ContrivedModel, self).__init__()
        self.first_dense = keras.layers.Dense(64)
        self.kernel = self.add_weight("kernel", shape=(64, 10))
        self.bias = self.add_weight("bias", shape=(10,))

    def call(self, inputs):
        x = self.first_dense(inputs)
        return tf.matmul(x, self.kernel) + self.bias


model = ContrivedModel()
# 在输入上调用模型以创建密集层的变量。
_ = model(tf.ones((1, 784)))

# 创建一个与之前结构相同的检查点，并加载权重。
tf.train.Checkpoint(
    dense=model.first_dense, kernel=model.kernel, bias=model.bias
).restore(ckpt_path).assert_consumed()

time: 325 ms (started: 2021-08-26 01:44:35 +08:00)


## HDF5格式
HDF5 格式包含按层名称分组的权重。 权重是通过将可训练权重列表连接到不可训练权重列表（与 layer.weights 相同）来排序的列表。 因此，如果模型具有与检查点中保存的相同的层和可训练状态，则该模型可以使用 hdf5 检查点。

例子：

In [36]:
# Runnable example
sequential_model = keras.Sequential(
    [
        keras.Input(shape=(784,), name="digits"),
        keras.layers.Dense(64, activation="relu", name="dense_1"),
        keras.layers.Dense(64, activation="relu", name="dense_2"),
        keras.layers.Dense(10, name="predictions"),
    ]
)
sequential_model.save_weights("weights.h5")
sequential_model.load_weights("weights.h5")

time: 134 ms (started: 2021-08-26 01:46:39 +08:00)


请注意，当模型包含嵌套层时，更改 layer.trainable 可能会导致不同的 layer.weights 排序。

In [37]:
class NestedDenseLayer(keras.layers.Layer):
    def __init__(self, units, name=None):
        super(NestedDenseLayer, self).__init__(name=name)
        self.dense_1 = keras.layers.Dense(units, name="dense_1")
        self.dense_2 = keras.layers.Dense(units, name="dense_2")

    def call(self, inputs):
        return self.dense_2(self.dense_1(inputs))


nested_model = keras.Sequential([keras.Input((784,)), NestedDenseLayer(10, "nested")])
variable_names = [v.name for v in nested_model.weights]
print("variables: {}".format(variable_names))

print("\nChanging trainable status of one of the nested layers...")
nested_model.get_layer("nested").dense_1.trainable = False

variable_names_2 = [v.name for v in nested_model.weights]
print("\nvariables: {}".format(variable_names_2))
print("variable ordering changed:", variable_names != variable_names_2)

variables: ['nested/dense_1/kernel:0', 'nested/dense_1/bias:0', 'nested/dense_2/kernel:0', 'nested/dense_2/bias:0']

Changing trainable status of one of the nested layers...

variables: ['nested/dense_2/kernel:0', 'nested/dense_2/bias:0', 'nested/dense_1/kernel:0', 'nested/dense_1/bias:0']
variable ordering changed: True
time: 77.6 ms (started: 2021-08-26 01:47:50 +08:00)


## 迁移学习示例
从 HDF5 加载预训练权重时，建议将权重加载到原始检查点模型中，然后将所需的权重/层提取到新模型中。

例子：

In [38]:
def create_functional_model():
    inputs = keras.Input(shape=(784,), name="digits")
    x = keras.layers.Dense(64, activation="relu", name="dense_1")(inputs)
    x = keras.layers.Dense(64, activation="relu", name="dense_2")(x)
    outputs = keras.layers.Dense(10, name="predictions")(x)
    return keras.Model(inputs=inputs, outputs=outputs, name="3_layer_mlp")


functional_model = create_functional_model()
functional_model.save_weights("pretrained_weights.h5")

# In a separate program:
pretrained_model = create_functional_model()
pretrained_model.load_weights("pretrained_weights.h5")

# Create a new model by extracting layers from the original model:
extracted_layers = pretrained_model.layers[:-1]
extracted_layers.append(keras.layers.Dense(5, name="dense_3"))
model = keras.Sequential(extracted_layers)
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                50240     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 325       
Total params: 54,725
Trainable params: 54,725
Non-trainable params: 0
_________________________________________________________________
time: 169 ms (started: 2021-08-26 01:49:03 +08:00)


In [39]:
extracted_layers

time: 3.06 ms (started: 2021-08-26 01:49:10 +08:00)
